In [1]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import skimage.draw
import matplotlib
import matplotlib.pyplot as plt
import json
import pandas as pd
from multiprocessing import freeze_support
from google.cloud import storage
from IPython.display import Image
from mrcnn import model as modellib, utils
from mrcnn.config import Config

In [2]:
# DON'T NEED TO RUN THIS AGAIN
# client = storage.Client()
# DATA_BUCKET = 'w207-final-project'
# databucket = client.get_bucket(DATA_BUCKET)
# datafiles = databucket.list_blobs()
# files=[a.name for a in datafiles if a.name.startswith('train_clean')]
# # files=[a.name for a in datafiles if a.name.startswith('val_clean')]
# files=[a.name for a in datafiles if a.name.startswith('test_clean')]
# # files=[a.name for a in datafiles if a.name.endswith('.json')]
# # print(files)
# Copy files from bucket into local folder
# for file in files:
#     blob = databucket.get_blob(file)
#     blob.download_to_filename(file)



# Check if all files are copied over
list = os.listdir('test_clean') # dir is your directory path
number_files = len(list)
print(number_files)

514


In [3]:
ROOT_DIR = os.path.abspath("")

# Path to trained weights file
# Original weights path
COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# New weights path
# COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "logs/object20220406T0321/mask_rcnn_object_0020.h5")

DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")

In [4]:

class CustomConfig(Config):
    """Configuration for training on the custom  dataset.
    Derives from the base Config class and overrides some values.
    """
    # Give the configuration a recognizable name
    NAME = "object"

    # We use a GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
    # ADJUSTED FROM IMAGES_PER_GPU = 2, MAY CHANGE BACK IF USING CLOUD
    IMAGES_PER_GPU = 1

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + drivable area
    # NUM_CLASSES = 1 + 2  # Background + drivable area + lane

    # Number of training steps per epoch
    # REMEMBER TO CHANGE AFTER TESTING
    STEPS_PER_EPOCH = 2
    # STEPS_PER_EPOCH = 10

    # Number of validation steps to run at the end of every training epoch.
    # A bigger number improves accuracy of validation stats, but slows
    # down the training.
    # START SMALLER?
    # VALIDATION_STEPS = 10
    VALIDATION_STEPS = 10
    
    # Supported values are: resnet50, resnet101.
    # You can also provide a callable that should have the signature
    # of model.resnet_graph. If you do so, you need to supply a callable
    # to COMPUTE_BACKBONE_SHAPE as well
    # BACKBONE = "resnet101" # Default
    BACKGONE = "resnet50"

    # Skip detections below % confidence specified
    # DETECTION_MIN_CONFIDENCE = 0.5
    DETECTION_MIN_CONFIDENCE = 0.9

In [5]:


class CustomDataset(utils.Dataset):

    def load_custom(self, dataset_dir, subset):

        """Load a subset of the driving dataset.
        dataset_dir: Root directory of the dataset.
        subset: Subset to load: train or val
        """

        # Add classes. We have only one class to add.
        self.add_class("object", 1, "drivable area")
        # self.add_class("object", 2, "lane")
        

        # Train or validation dataset?
        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir, subset + "_clean")
        # dataset_dir = os.path.join(dataset_dir, subset + "_test")

        # Given an image (i.e. a row in train_labels_split)
        # Get all the items that are considered drivable area
        # For each item, save the x and y vertices into two lists
        
        if subset == 'train':
            labels = train_labels_cleaned
        else:
            labels = val_labels_cleaned
            
        for index, image in labels.iterrows():

            polygons = []
            objects = []
            for obj in image['labels']:
                x = []
                y = []
                if obj['category'] == 'drivable area':
                    for coord in obj['poly2d'][0]['vertices']:
                        if coord[0] >= 1280:
                            x.append(1279)
                        else:
                            x.append(coord[0])
                        if coord[1] >= 720:
                            y.append(719)
                        else:
                            y.append(coord[1])
                    polygons.append({'name': 'polygon',
                                      'all_points_x': x,
                                      'all_points_y': y})
                    objects.append('drivable area')
                # elif obj['category'] == 'lane':
                #     for coord in obj['poly2d'][0]['vertices']:
                #         if coord[0] >= 1280:
                #             x.append(1279)
                #         else:
                #             x.append(coord[0])
                #         if coord[1] >= 720:
                #             y.append(719)
                #         else:
                #             y.append(coord[1])
                #     polygons.append({'name': 'polygon',
                #                       'all_points_x': x,
                #                       'all_points_y': y})
                #     objects.append('lane')
                    
            image_path = os.path.join(dataset_dir, image['name'])

            # Only use commented code if trying out drivable area and lane. Use the uncommented code for just drivable area evaluation
            name_dict = {"drivable area": 1}
            # name_dict = {"drivable area": 1,"lane": 2}
            
            num_ids = [name_dict[a] for a in objects]

            self.add_image(
                "object",  ## for a single class just add the name here
                image_id=image['name'],  # use file name as a unique image id
                path=image_path,
                width=1280, height=720,
                polygons=polygons,
                num_ids=num_ids
                )
        return

    def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        # If not a Dog-Cat dataset image, delegate to parent class.
        image_info = self.image_info[image_id]
        if image_info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)

        # Convert polygons to a bitmap mask of shape
        # [height, width, instance_count]
        info = self.image_info[image_id]
        if info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)
        num_ids = info['num_ids']
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):
            # Get indexes of pixels inside the polygon and set them to 1
        	rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])

        	mask[rr, cc, i] = 1

        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID only, we return an array of 1s
        # Map class names to class IDs.
        num_ids = np.array(num_ids, dtype=np.int32)
        return mask, num_ids #np.ones([mask.shape[-1]], dtype=np.int32)

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "object":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)

In [6]:
def train(model):
    """Train the model."""
    # Training dataset.
    dataset_train = CustomDataset()
    dataset_train.load_custom(ROOT_DIR, "train")
    dataset_train.prepare()

    # Validation dataset
    dataset_val = CustomDataset()
    dataset_val.load_custom(ROOT_DIR, "val")
    dataset_val.prepare()

    # *** This training schedule is an example. Update to your needs ***
    # Since we're using a very small dataset, and starting from
    # COCO trained weights, we don't need to train too long. Also,
    # no need to train all layers, just the heads should do it.
    print("Training network heads")

    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=40,
                layers='heads')



In [7]:
# Load JSON files for image segment labels

with open('test/train_clean/bdd100k_labels_images_train.json') as json_file:
    train_labels = json.load(json_file)

with open('val_clean/bdd100k_labels_images_val.json') as json_file:
    val_labels = json.load(json_file)

In [9]:

# REQUIRED: Normalize semi-structured JSON data into a pd dataframe
# Note that train_labels json file contains both labels for training and validation set 
train_labels_normalized = pd.json_normalize(train_labels)

train_filenames = []
val_filenames = []

# REMEMBER TO REPLACE AFTER INITIAL TESTING
for myfile in os.listdir(ROOT_DIR + "/train_clean"):
    if myfile.endswith(".jpg"):
        train_filenames.append(os.path.join(myfile))

for myfile in os.listdir(ROOT_DIR + "/val_clean"):
    if myfile.endswith(".jpg"):
        val_filenames.append(os.path.join(myfile))

# Save labels for only the images within the folder contents
train_labels_cleaned = train_labels_normalized.loc[train_labels_normalized['name'].isin(train_filenames)]

# val_labels_split
val_labels_cleaned = train_labels_normalized.loc[train_labels_normalized['name'].isin(val_filenames)]



In [11]:
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
gpus
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)
    print("error")

1 Physical GPUs, 1 Logical GPU


In [12]:

config = CustomConfig()
model = modellib.MaskRCNN(mode="training", config=config,
                                  model_dir=DEFAULT_LOGS_DIR)

weights_path = COCO_WEIGHTS_PATH
        # Download weights file
if not os.path.exists(weights_path):
  utils.download_trained_weights(weights_path)

model.load_weights(weights_path, by_name=True, exclude=[
            "mrcnn_class_logits", "mrcnn_bbox_fc",
            "mrcnn_bbox", "mrcnn_mask"])


2022-04-09 17:54:34.351727: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-09 17:54:34.352501: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-09 17:54:34.353313: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-09 17:54:34.354028: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-09 17:54:34.354711: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from S

In [13]:
train(model)

Training network heads

Starting at epoch 0. LR=0.001

Checkpoint Path: /home/jupyter/logs/object20220409T1754/mask_rcnn_object_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
rpn_model              (Functional)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask_conv4       (TimeDistributed)
mrcnn_mask_bn4         (TimeDistributed)
mrcnn_bbox_fc          (TimeDistributed)
mrcnn_

/opt/conda/lib/python3.7/site-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)
2022-04-09 17:54:46.678064: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-09 17:54:46.678482: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-09 17:54:46.678759: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-09 17:54:46.679086: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative 

Epoch 1/40


/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("training/SGD/gradients/gradients/roi_align_classifier/concat_grad/sub:0", shape=(None,), dtype=int32), values=Tensor("training/SGD/gradients/gradients/roi_align_classifier/concat_grad/GatherV2_2:0", shape=(None, 7, 7, 256), dtype=float32), dense_shape=Tensor("training/SGD/gradients/gradients/roi_align_classifier/concat_grad/Shape:0", shape=(4,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("training/SGD/gradients/gradients/roi_align_classifier/concat_grad/sub_1:0", shape=(None,), dtype=int32), values=Tensor("training/SGD/gradients/gradients/roi_align_cl

1/2 [==============>...............] - ETA: 18s - batch: 0.0000e+00 - size: 1.0000 - loss: 37.1341 - rpn_class_loss: 15.8546 - rpn_bbox_loss: 12.3023 - mrcnn_class_loss: 8.9772 - mrcnn_bbox_loss: 0.0000e+00 - mrcnn_mask_loss: 0.0000e+00

2022-04-09 17:55:35.950865: I tensorflow/core/profiler/lib/profiler_session.cc:110] Profiler session initializing.
2022-04-09 17:55:35.950903: I tensorflow/core/profiler/lib/profiler_session.cc:125] Profiler session started.
2022-04-09 17:55:35.952423: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1630] Profiler found 1 GPUs
2022-04-09 17:55:35.952928: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcupti.so.11.2'; dlerror: libcupti.so.11.2: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2022-04-09 17:55:36.964370: I tensorflow/core/profiler/lib/profiler_session.cc:67] Profiler session collecting data.
2022-04-09 17:55:36.965285: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1764] CUPTI activity buffer flushed
2022-04-09 17:55:37.033620: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:521]  GpuTracer

2/2 [==============================] - ETA: 0s - batch: 0.5000 - size: 1.0000 - loss: 27.0627 - rpn_class_loss: 11.6807 - rpn_bbox_loss: 10.8933 - mrcnn_class_loss: 4.4886 - mrcnn_bbox_loss: 0.0000e+00 - mrcnn_mask_loss: 0.0000e+00     

/opt/conda/lib/python3.7/site-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates
2022-04-09 17:55:41.340053: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:690] Error in PredictCost() for the op: op: "CropAndResize" attr { key: "T" value { type: DT_FLOAT } } attr { key: "extrapolation_value" value { f: 0 } } attr { key: "method" value { s: "bilinear" } } inputs { dtype: DT_FLOAT shape { dim { size: -395 } dim { size: 56 } dim { size: 56 } dim { size: 1 } } } inputs { dtype: DT_FLOAT shape { dim { size: -12 } dim { size: 4 } } } inputs { dtype: DT_INT32 shape { dim { size: -12 } } } inputs { dtype: DT_INT32 shape { dim { size: 2 } } value { dtype: DT_INT32 tensor_shape { dim { size: 2 } } int_val: 28 } } device { type: "GPU" vendor: "NVIDIA" model: "Tesla T4" frequency: 1590 num_cores: 40 e

2/2 [==============================] - 50s 31s/step - batch: 0.5000 - size: 1.0000 - loss: 27.0627 - rpn_class_loss: 11.6807 - rpn_bbox_loss: 10.8933 - mrcnn_class_loss: 4.4886 - mrcnn_bbox_loss: 0.0000e+00 - mrcnn_mask_loss: 0.0000e+00 - val_loss: 6.8463 - val_rpn_class_loss: 0.9560 - val_rpn_bbox_loss: 5.8902 - val_mrcnn_class_loss: 0.0000e+00 - val_mrcnn_bbox_loss: 0.0000e+00 - val_mrcnn_mask_loss: 0.0000e+00
Epoch 2/40
2/2 [==============================] - 6s 6s/step - batch: 0.5000 - size: 1.0000 - loss: 9.8695 - rpn_class_loss: 0.6442 - rpn_bbox_loss: 4.8844 - mrcnn_class_loss: 0.0250 - mrcnn_bbox_loss: 3.8452 - mrcnn_mask_loss: 0.4706 - val_loss: 9.2958 - val_rpn_class_loss: 0.2353 - val_rpn_bbox_loss: 4.5947 - val_mrcnn_class_loss: 0.0502 - val_mrcnn_bbox_loss: 3.5944 - val_mrcnn_mask_loss: 0.8213
Epoch 3/40
2/2 [==============================] - 9s 8s/step - batch: 0.5000 - size: 1.0000 - loss: 7.9673 - rpn_class_loss: 0.2550 - rpn_bbox_loss: 5.9525 - mrcnn_class_loss: 0.0144